# Application of Deep Learning Algorithms for S&P 500 Stock Prices Prediction

## Imports

In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Explorative Data Analysis

### Parsing
Historical Data of the S&P 500 index were downloaded from: https://www.nasdaq.com/de/market-activity/index/spx/historical.

In [78]:
# Read in CSV from directory
df = pd.read_csv('data/historical_data_sp_500.csv', sep=',', parse_dates=['Datum'], index_col='Datum')

In [79]:
# Verify shape of dataframe
df.shape

(2554, 5)

In [80]:
# Rename index column
df.index.names = ['date']


In [81]:
# Rename feature columns
df.rename(columns={'Schluss/Letzter':'close', 'Volumen':'volume', 'Eröffnungskurs':'open', 'Hoch':'high', 'Tief':'low'}, inplace=True)

In [82]:
# Show first few rows
df.head(n=5)

,close,volume,open,high,low
date,,,,,
2022-01-03,4796.56,--,4778.14,4796.64,4758.17
2021-12-31,4766.18,--,4775.21,4786.83,4765.75
2021-12-30,4778.73,--,4794.23,4808.93,4775.33
2021-12-29,4793.06,--,4788.64,4804.06,4778.08
2021-12-28,4786.35,--,4795.49,4807.02,4780.04


In [83]:
# Show last few rows
df.tail(n=5)

,close,volume,open,high,low
date,,,,,
2012-01-10,1292.08,--,1280.77,1296.46,1280.77
2012-01-09,1280.70,--,1277.83,1281.99,1274.55
2012-01-06,1277.81,--,1280.93,1281.84,1273.34
2012-01-05,1281.06,--,1277.30,1283.05,1265.26
2012-01-04,1277.30,--,1277.03,1278.73,1268.10


In [84]:
# It seems that the latest dates are on top of the dataframe. However, we want it to be reverse
df = df.iloc[::-1]

In [85]:
df.head(n=5)

,close,volume,open,high,low
date,,,,,
2012-01-04,1277.30,--,1277.03,1278.73,1268.10
2012-01-05,1281.06,--,1277.30,1283.05,1265.26
2012-01-06,1277.81,--,1280.93,1281.84,1273.34
2012-01-09,1280.70,--,1277.83,1281.99,1274.55
2012-01-10,1292.08,--,1280.77,1296.46,1280.77


In [86]:
df.tail(n=5)

,close,volume,open,high,low
date,,,,,
2021-12-28,4786.35,--,4795.49,4807.02,4780.04
2021-12-29,4793.06,--,4788.64,4804.06,4778.08
2021-12-30,4778.73,--,4794.23,4808.93,4775.33
2021-12-31,4766.18,--,4775.21,4786.83,4765.75
2022-01-03,4796.56,--,4778.14,4796.64,4758.17


### Verification of the data types

In [87]:
df.dtypes

close     float64
volume     object
open      float64
high      float64
low       float64
dtype: object

In [88]:
# The volume is not present for any row --> let's remove this column
df = df.drop('volume', axis=1)

### Analysis of duplicates

In [89]:
df.duplicated().any()

True

In [90]:
df[df.duplicated(keep=False)].head(n=10)

,close,open,high,low
date,,,,
2012-10-29,1411.94,0.0,0.0,0.0
2012-10-30,1411.94,0.0,0.0,0.0


In [91]:
# Show these two data points --> they have both missing values
df.loc['2012-10-28':'2012-10-30']

,close,open,high,low
date,,,,
2012-10-29,1411.94,0.0,0.0,0.0
2012-10-30,1411.94,0.0,0.0,0.0


### Analysis of missing values

In [92]:
# We can see that there are no null values
df.isna().any()

close    False
open     False
high     False
low      False
dtype: bool

In [93]:
(df['close'] != 0).all()

True

In [94]:
(df['open'] != 0).all()

False

In [95]:
(df['high'] != 0).all()

False

In [96]:
(df['low'] != 0).all()

False

In [97]:
df.loc[(df['open'] == 0) & (df['high'] == 0) & (df['low'] == 0)]

,close,open,high,low
date,,,,
2012-01-16,1289.09,0.0,0.0,0.0
2012-02-20,1361.23,0.0,0.0,0.0
2012-04-06,1398.08,0.0,0.0,0.0
2012-05-28,1317.82,0.0,0.0,0.0
2012-07-04,1374.02,0.0,0.0,0.0
2012-09-03,1406.58,0.0,0.0,0.0
2012-10-29,1411.94,0.0,0.0,0.0
2012-10-30,1411.94,0.0,0.0,0.0
2012-11-22,1391.03,0.0,0.0,0.0


In [105]:
# Interpolate the zero values
df.replace(0, np.NaN, inplace=True)

In [122]:
df['open'] = df['open'].interpolate(method='linear', limit_direction='forward')

In [123]:
df['high'] = df['high'].interpolate(method='linear', limit_direction='forward')

In [124]:
df['low'] = df['low'].interpolate(method='linear', limit_direction='forward')

In [127]:
df.isna().any()

close    False
open     False
high     False
low      False
dtype: bool